# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys2 import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('output_data/city_data.csv')
city_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (F)
0,Chokurdakh,70.63,147.92,279.83,57,6,5.56,RU,1591829852,44.02
1,Alofi,-19.06,-169.92,299.15,83,75,1.50,NU,1591829852,78.80
2,Philadelphia,39.95,-75.16,304.26,58,75,7.70,US,1591829588,88.00
3,Vaini,-21.20,-175.20,298.15,94,75,5.70,TO,1591829853,77.00
4,Flinders,-34.58,150.86,286.48,97,49,0.45,AU,1591829629,55.99
...,...,...,...,...,...,...,...,...,...,...
549,Bakchar,57.02,82.07,285.57,85,35,3.44,RU,1591829982,54.36
550,Ubari,26.59,12.78,307.29,22,63,8.37,LY,1591829983,93.45
551,Brownsville,25.90,-97.50,303.15,79,20,6.70,US,1591829778,86.00
552,Kupang,-10.17,123.58,298.21,77,47,5.41,ID,1591829983,77.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_data["Humidity"]
humidity

0      57
1      83
2      58
3      94
4      97
       ..
549    85
550    22
551    79
552    77
553    51
Name: Humidity, Length: 554, dtype: int64

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_data_narrow = city_data.loc[city_data['Max Temp']>294]

city_data_narrow = city_data_narrow.loc[city_data['Max Temp']<300]

city_data_narrow = city_data_narrow.loc[city_data['Wind Speed']<10]

city_data_narrow = city_data_narrow.loc[city_data['Cloudiness']==0]

city_data_narrow.dropna(how='any')

city_data_narrow

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (F)
51,Jardim,-21.48,-56.14,297.68,69,0,2.25,BR,1591829863,76.15
77,Vila Velha,-20.33,-40.29,299.15,65,0,2.60,BR,1591829665,78.80
92,Morondava,-20.28,44.28,294.86,55,0,6.18,MG,1591829872,71.08
101,Shkurinskaya,46.59,39.36,294.68,74,0,3.50,RU,1591829874,70.75
113,Atyrau,47.12,51.88,296.15,27,0,4.00,KZ,1591829877,73.40
124,Caravelas,-17.71,-39.25,295.70,93,0,2.55,BR,1591829880,72.59
127,Beyneu,45.32,55.20,299.71,19,0,3.22,KZ,1591829881,79.81
139,São Filipe,14.90,-24.50,297.41,84,0,2.47,CV,1591829884,75.67
143,Lompoc,34.64,-120.46,299.26,49,0,6.70,US,1591829847,79.00
344,Flagstaff,35.20,-111.65,299.82,2,0,1.34,US,1591829893,80.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
city_data_narrow["Hotel Name"] = ""

hotel_df = city_data_narrow
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (F),Hotel Name
51,Jardim,-21.48,-56.14,297.68,69,0,2.25,BR,1591829863,76.15,
77,Vila Velha,-20.33,-40.29,299.15,65,0,2.60,BR,1591829665,78.80,
92,Morondava,-20.28,44.28,294.86,55,0,6.18,MG,1591829872,71.08,
101,Shkurinskaya,46.59,39.36,294.68,74,0,3.50,RU,1591829874,70.75,
113,Atyrau,47.12,51.88,296.15,27,0,4.00,KZ,1591829877,73.40,
124,Caravelas,-17.71,-39.25,295.70,93,0,2.55,BR,1591829880,72.59,
127,Beyneu,45.32,55.20,299.71,19,0,3.22,KZ,1591829881,79.81,
139,São Filipe,14.90,-24.50,297.41,84,0,2.47,CV,1591829884,75.67,
143,Lompoc,34.64,-120.46,299.26,49,0,6.70,US,1591829847,79.00,
344,Flagstaff,35.20,-111.65,299.82,2,0,1.34,US,1591829893,80.01,


In [7]:
##----- Search of hotels near to each city (no more than 5 km)------## 

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f'{lat}  ,  {lng}')
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        print(f"Hotel found nearby. {index}")
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping. {index}")

-21.48  ,  -56.14
Hotel found nearby. 51
-20.33  ,  -40.29
Hotel found nearby. 77
-20.28  ,  44.28
Hotel found nearby. 92
46.59  ,  39.36
Missing field/result... skipping. 101
47.12  ,  51.88
Hotel found nearby. 113
-17.71  ,  -39.25
Hotel found nearby. 124
45.32  ,  55.2
Hotel found nearby. 127
14.9  ,  -24.5
Hotel found nearby. 139
34.64  ,  -120.46
Hotel found nearby. 143
35.2  ,  -111.65
Hotel found nearby. 344
-20.44  ,  -53.76
Hotel found nearby. 417
31.61  ,  34.76
Hotel found nearby. 420
38.44  ,  57.43
Missing field/result... skipping. 453
30.15  ,  62.6
Missing field/result... skipping. 510
14.92  ,  -23.51
Hotel found nearby. 531
-15.9  ,  -52.25
Hotel found nearby. 537


In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (F),Hotel Name
51,Jardim,-21.48,-56.14,297.68,69,0,2.25,BR,1591829863,76.15,Vitória Hotel
77,Vila Velha,-20.33,-40.29,299.15,65,0,2.60,BR,1591829665,78.80,Golden Tulip Porto Vitória
92,Morondava,-20.28,44.28,294.86,55,0,6.18,MG,1591829872,71.08,Palissandre Cote Ouest resort & SPA
101,Shkurinskaya,46.59,39.36,294.68,74,0,3.50,RU,1591829874,70.75,
113,Atyrau,47.12,51.88,296.15,27,0,4.00,KZ,1591829877,73.40,Renaissance Atyrau Hotel
124,Caravelas,-17.71,-39.25,295.70,93,0,2.55,BR,1591829880,72.59,Pousada Liberdade
127,Beyneu,45.32,55.20,299.71,19,0,3.22,KZ,1591829881,79.81,"HOTEL ""AKNUR"""
139,São Filipe,14.90,-24.50,297.41,84,0,2.47,CV,1591829884,75.67,Hotel Xaguate
143,Lompoc,34.64,-120.46,299.26,49,0,6.70,US,1591829847,79.00,Embassy Suites by Hilton Lompoc Central Coast
344,Flagstaff,35.20,-111.65,299.82,2,0,1.34,US,1591829893,80.01,Little America Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [10]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Add marker layer ontop of heat map
marker_locations = locations2
markers = gmaps.marker_layer(marker_locations,info_box_content =hotel_info)
fig.add_layer(markers)


# Display figure
fig



Figure(layout=FigureLayout(height='420px'))